In [ ]:
using VideoIO
using Images, ColorTypes
using NativeFileDialog: pick_file

function find_buoy_position(frame::AbstractArray)
    
    # Convert the frame to grayscale for easier processing
    gray_frame = channelview(colorview(Gray, frame))

    # Thresholding based on intensity (adjust the threshold for your video)
    buoy_mask = gray_frame .> 0.7  # Set threshold based on buoy's brightness

    # Find connected components (blobs)
    labels = label_components(buoy_mask)

    # Calculate the centroid of the largest blob
    largest_blob, largest_size = argmax(map(label -> sum(labels .== label), unique(labels)))
    
    if largest_size > 10  # Ensure blob is large enough to be the buoy
        buoy_y, buoy_x = mean(findall(labels .== largest_blob), dims=2)  # Center coordinates
        return((buoy_x[1], buoy_y[1]))  # X, Y coordinates of the buoy
    else
        return(nothing)  # No buoy detected
    end
    
end


infil = pick_file()

# Open the video and initialize position storage
video = VideoIO.openvideo(infil)
positions = []

# Process frames with optional sampling
frame_counter = 0

while !eof(video)
    
    frame = VideoIO.read(video)
    frame_counter += 1

    # Sample every nth frame (e.g., every 10th frame for a smoother, faster run)
    if frame_counter % 10 == 0
        buoy_position = find_buoy_position(frame)
        if buoy_position !== nothing
            push!(positions, buoy_position)
        end
    end
    
end

VideoIO.close(video)


In [ ]:
using VideoIO
using NativeFileDialog: pick_file

infil = pick_file()

# Define the video file path
video_file = infil

# Construct the ffmpeg command
ffmpeg_cmd = `ffmpeg -i $video_file`

# Run the command and capture the output
try
    run(ffmpeg_cmd)
catch e
    println("Error encountered: $e")
end


In [ ]:
using VideoIO
using Images, ColorTypes
using NativeFileDialog: pick_file

function find_buoy_position(frame)
    println("here")
    gray_frame = channelview(colorview(Gray, frame))
    buoy_mask = gray_frame .> 0.7
    labels = label_components(buoy_mask)
    largest_blob, largest_size = argmax(map(label -> sum(labels .== label), unique(labels)))

    if largest_size > 10
        buoy_y, buoy_x = mean(findall(labels .== largest_blob), dims=2)
        return (buoy_x[1], buoy_y[1])
    else
        return nothing
    end
end

infil = pick_file()
video_reader = VideoIO.openvideo(infil)  # Open video as VideoReader

frame_limit = 50
positions = []

for i in 1:frame_limit

    println(i)
    if eof(video_reader)
        println("Reached the end of the video before frame limit.")
        break
    end

    frame = VideoIO.read(video_reader)

    if i % 10 == 0
        println("Processing frame $i")
        buoy_position = find_buoy_position(frame)
        println("here too")
        if buoy_position !== nothing
            push!(positions, buoy_position)
        end
    end

    
end

VideoIO.close(video_reader)